# Analysis of Earthcam data with selfmade CNN

In [4]:
import torch
import torch.nn as nn

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## Import Trained CNN model and its weights

In [5]:
# Define CNN class here
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Define layers

    def forward(self, x):
        # Define forward pass
        return x


In [9]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(BasicBlock, self).__init__()
        # Define layers

    def forward(self, x):
        # Define forward pass
        return x

In [10]:
# Define the path to model
model_path = '/content/drive/MyDrive/CNN_Model/cnn_model_complete.pth'

model = torch.load(model_path, map_location=torch.device('cpu'))
model.eval()


CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1

## Preparing Data

In [12]:
import os
from PIL import Image
from torchvision import transforms

# Define paths to image folders
folder_paths = {
    'NewYork': '/content/drive/MyDrive/Earthcam Images/downloaded_images/Earthcam_NewYork',
    'NewOrleans': '/content/drive/MyDrive/Earthcam Images/downloaded_images/Earthcam_NewOrleans',
    'London': '/content/drive/MyDrive/Earthcam Images/downloaded_images/Earthcam_London'
}

# Specify the years of interest
years = ['2019', '2020', '2022']

# Define image preprocessing steps
preprocess = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


## Analyze Images

In [13]:
import torch.nn.functional as F

# Initialize a dictionary to store results
results = {'2019': [], '2020': [], '2022': []}

# Function to predict using the model
def predict_image(image):
    image = preprocess(image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        output = model(image)
        probabilities = F.softmax(output[0], dim=0)
    return probabilities

# Analyze images
for city, path in folder_paths.items():
    for filename in os.listdir(path):
        if filename.endswith(".jpg"):
            year = filename.split('_')[1].split('-')[-1]
            if year in years:
                image_path = os.path.join(path, filename)
                image = Image.open(image_path).convert('RGB')
                probabilities = predict_image(image)

                # Store the probability for the relevant year
                results[year].append(probabilities)

# Convert results to more interpretable data (e.g., average probability)
average_results = {year: torch.stack(results[year]).mean(dim=0).item() for year in results}

print(average_results)


RuntimeError: a Tensor with 49152 elements cannot be converted to Scalar

## Results

In [ ]:
import matplotlib.pyplot as plt

# Plot the average probabilities
plt.bar(average_results.keys(), average_results.values())
plt.title('Average Probability of People on Streets Over Years')
plt.xlabel('Year')
plt.ylabel('Average Probability')
plt.show()
